In this notebook, we will be generating features to use for PIPE.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
masks_path = "data/yeast_masks_singlesite_filtered.pkl"
uniprot_file = "data/uniprot-proteome UP000002311.tab"# reference proteome at https://www.uniprot.org/proteomes/UP000002311
pssm_path = "data/yeast_pssms/"

processed_data_path = "data/yeast_processed/"

# Load data and masks

## Load masks

In [3]:
ppi_masks = pd.read_pickle(masks_path)
ppi_masks.head()

,Uniprot ID A,Uniprot ID B,Domain_id_a,Domain_id_b,Domain positions A,Domain positions B,Sites Masks
0,P53081,P53081,[PF01784],[PF01784],"[(12, 275)]","[(12, 275)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,Q12188,P47037,[PF04825],[PF02463],"[(11, 136)]","[(2, 1208)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Q03020,P25374,[PF01592],[PF00266],"[(34, 161)]","[(100, 462)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,P17065,P51996,[PF06428],[PF00071],"[(79, 170)]","[(15, 176)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,P00546,P20437,[PF00069],[PF00134],"[(8, 295)]","[(41, 193)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [4]:
ppi_masks.shape

(1606, 7)

In [5]:
# All the protein sequences we need information for
proteins = ppi_masks['Uniprot ID A'].append(ppi_masks['Uniprot ID B'], ignore_index=True).unique()
proteins, proteins.shape

(array(['P53081', 'Q12188', 'Q03020', ..., 'P22219', 'P12612', 'P38080'],
       dtype=object),
 (1258,))

## Load sequences

In [6]:
uniprot_df = pd.read_csv(uniprot_file,
                        sep = "\t", index_col='Entry')
print("Loaded UniProt proteome")
uniprot_df

Loaded UniProt proteome


,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence
Entry,,,,,,,
P21192,ACE2_YEAST,reviewed,Metallothionein expression activator,ACE2 YLR131C L3123 L9606.10,Saccharomyces cerevisiae (strain ATCC 204508 /...,770,MDNVVDPWYINPSGFAKDTQDEEYVQHHDNVNPTIPPPDNYILNNE...
P46993,ASG7_YEAST,reviewed,Protein ASG7 (A-specific gene 7 protein),ASG7 YJL170C J0514,Saccharomyces cerevisiae (strain ATCC 204508 /...,209,MTTLASSIEHKTKHLAAPFENDENPWMKKYCCQCKSCKMSVPVQPW...
P47117,ARP3_YEAST,reviewed,Actin-related protein 3 (Actin-like protein AR...,ARP3 ACT4 YJR065C J1760,Saccharomyces cerevisiae (strain ATCC 204508 /...,449,MSYLNNPAVVMDNGTGLTKLGFAGNDSPSWVFPTAIATAAPSNTKK...
P22768,ASSY_YEAST,reviewed,Argininosuccinate synthase (EC 6.3.4.5) (Citru...,ARG1 YOL058W O1228,Saccharomyces cerevisiae (strain ATCC 204508 /...,420,MSKGKVCLAYSGGLDTSVILAWLLDQGYEVVAFMANVGQEEDFDAA...
P29311,BMH1_YEAST,reviewed,Protein BMH1,BMH1 YER177W,Saccharomyces cerevisiae (strain ATCC 204508 /...,267,MSTSREDSVYLAKLAEQAERYEEMVENMKTVASSGQELSVEERNLL...
...,...,...,...,...,...,...,...
P47049,UBX6_YEAST,reviewed,UBX domain-containing protein 6,UBX6 YJL048C J1164,Saccharomyces cerevisiae (strain ATCC 204508 /...,396,MYEMSGIDSLFHDRVVHDYSHTSEQVIVVYISSAAGDNSWLHQWFK...
P53142,VPS73_YEAST,reviewed,Vacuolar protein sorting-associated protein 73,VPS73 YGL104C G3090,Saccharomyces cerevisiae (strain ATCC 204508 /...,486,MNRILSSASLLSNVSMPRQNKHKITKALCYAIIVASIGSIQFGYHL...
Q05919,VPS38_YEAST,reviewed,Vacuolar protein sorting-associated protein 38,VPS38 VPL17 YLR360W L8039.11,Saccharomyces cerevisiae (strain ATCC 204508 /...,439,MKRFLLSRRQRHLRMICFHNISLFRANGDSKLIKEYGDGFIPCFFI...


## ProtDCal

Generated using https://protdcal.zmb.uni-due.de/pages/form.php

## PSSMs

psiblast -query 'uniprot-proteome UP000002311.fasta' -db ~/sysc4906/s/swissprot -inclusion_ethresh 0.001 -num_iterations 2 -out_ascii_pssm yeast_filtered.pssm -save_pssm_after_last_round

/home/wma/sysc4906/ncbi-blast-2.11.0+/bin

for file in *.fasta; do echo "./runPsiBlast.sh '$file'"; done >> blastall.sh

rename -n 's/sp\|(.*?)\|.*/$1.pssm/' *

In [7]:
# psiblast -query .\output_sequences.fasta -db nr -out yeast_filtered_psiblast_out -evalue 0.001 -num_iterations 3 -out_pssm yeast_filtered_pssm_checkpoint -out_ascii_pssm yeast_filtered_pssm

In [8]:
AA = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

In [9]:
print("Loading PSSMs")
PSSMs = {}
for swissprot_id in proteins:
    pssm_A = pd.read_csv(pssm_path + swissprot_id + ".pssm", sep = "\s+", skiprows=2).reset_index()[AA].dropna()
    PSSMs[swissprot_id] = pssm_A.to_numpy()

Loading PSSMs


In [10]:
PSSM_masks = []
print("Generating PSSM masks")
for UA, UB, SM in zip(ppi_masks['Uniprot ID A'], ppi_masks['Uniprot ID B'], ppi_masks['Sites Masks']):
    try:
#         lengths = (uniprot_df['Length'].loc[UA], uniprot_df['Length'].loc[UB])
        
        # number according to position    
        seqA = PSSMs[UA]
        seqB = PSSMs[UB]
        
#         # normalize
#         seqA = np.divide(seqA, lengths[0])
#         seqB = np.divide(seqB, lengths[1])
        
        # Do a dot product for each 20 length vector along each position
        mask = np.matmul(seqA, seqB.T)
        
        assert SM.shape == mask.shape
        
        PSSM_masks.append(mask)
    except KeyError as inst:
        print(UA, UB)
        print(f"No uniprot entry found for protein {inst.args}")
        position_landscape.append(np.NaN)

Generating PSSM masks


In [11]:
PSSM_masks[:10]

[array([[118., -11., -15., ...,  -3.,  89.,  18.],
        [-11., 550.,  59., ...,  47.,   3.,  34.],
        [-15.,  59., 399., ..., 172., -11.,  28.],
        ...,
        [ -3.,  47., 172., ..., 138., -11.,  17.],
        [ 89.,   3., -11., ..., -11., 122.,  33.],
        [ 18.,  34.,  28., ...,  17.,  33.,  32.]]),
 array([[174.,  68., 113., ...,  14., -17.,  60.],
        [ 84.,   9.,  73., ...,  38.,  18.,  33.],
        [ 80.,   4., 120., ...,  45.,  57.,  18.],
        ...,
        [172.,  49., 179., ...,  32., -15.,  92.],
        [ 57., 116.,  41., ...,  -8., -17.,  16.],
        [ 84.,   9.,  73., ...,  38.,  18.,  33.]]),
 array([[ 81.,  82.,   6., ..., -10., -19.,  -8.],
        [ 87., 117.,   9., ...,  -4., -22.,  -3.],
        [ -4.,   7.,  52., ...,  34.,  33.,  42.],
        ...,
        [ 80., 101.,   1., ...,  -1., -26., -19.],
        [ 93., 129.,   3., ...,  -9., -23.,  -3.],
        [-10., -18.,  37., ...,  47.,  31.,  24.]]),
 array([[ 1.140e+02, -7.000e+00, -2.7

In [12]:
ppi_masks['PSSM_masks'] = PSSM_masks
ppi_masks

,Uniprot ID A,Uniprot ID B,Domain_id_a,Domain_id_b,Domain positions A,Domain positions B,Sites Masks,PSSM_masks
0,P53081,P53081,[PF01784],[PF01784],"[(12, 275)]","[(12, 275)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[118.0, -11.0, -15.0, 5.0, 33.0, 11.0, -8.0, ..."
1,Q12188,P47037,[PF04825],[PF02463],"[(11, 136)]","[(2, 1208)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[174.0, 68.0, 113.0, 28.0, -10.0, 152.0, 14.0..."
2,Q03020,P25374,[PF01592],[PF00266],"[(34, 161)]","[(100, 462)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[81.0, 82.0, 6.0, -8.0, 43.0, 56.0, 8.0, -9.0..."
3,P17065,P51996,[PF06428],[PF00071],"[(79, 170)]","[(15, 176)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[114.0, -7.0, -27.0, -40.0, -32.0, -22.0, -28..."
4,P00546,P20437,[PF00069],[PF00134],"[(8, 295)]","[(41, 193)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[164.0, -40.0, 3.0, -38.0, -37.0, 120.0, -32...."
...,...,...,...,...,...,...,...,...
1601,Q12403,P54837,[PF01105],[PF01105],"[(23, 218)]","[(20, 206)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[148.0, 17.0, 28.0, 80.0, 33.0, 56.0, 43.0, 8..."
1602,Q06549,Q06549,[PF00383],[PF00383],"[(8, 115)]","[(8, 115)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[111.0, -20.0, 32.0, -3.0, -26.0, 134.0, -35...."
1603,Q12335,Q12335,[PF03358],[PF03358],"[(8, 148)]","[(8, 148)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[206.0, 18.0, 25.0, 140.0, 75.0, 141.0, 132.0..."
1604,P53725,P38792,[PF10175],[PF15985],"[(9, 186)]","[(197, 239)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[71.0, -10.0, -17.0, 61.0, 70.0, 10.0, 70.0, ..."


## Position

In [13]:
position_landscape = []
print("Generating position feature landscape")
for UA, UB, SM in zip(ppi_masks['Uniprot ID A'], ppi_masks['Uniprot ID B'], ppi_masks['Sites Masks']):
    try:
        lengths = (uniprot_df['Length'].loc[UA], uniprot_df['Length'].loc[UB])
        
        # number according to position    
        seqA = np.arange(1, lengths[0]+1)[np.newaxis]
        seqB = np.arange(1, lengths[1]+1)[np.newaxis]
        
        # normalize
        seqA = np.divide(seqA, lengths[0])
        seqB = np.divide(seqB, lengths[1])
        
        mask = np.matmul(seqA.T, seqB)
        
        assert SM.shape == mask.shape
        
        position_landscape.append(mask)
    except KeyError as inst:
        print(UA, UB)
        print(f"No uniprot entry found for protein {inst.args}")
        position_landscape.append(np.NaN)
        
position_landscape = np.asarray(position_landscape)

Generating position feature landscape


C:\Users\mawil\Anaconda3\envs\SYSC4907\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [14]:
position_landscape[:2]

array([array([[1.20563272e-05, 2.41126543e-05, 3.61689815e-05, ...,
        3.44810957e-03, 3.46016590e-03, 3.47222222e-03],
       [2.41126543e-05, 4.82253086e-05, 7.23379630e-05, ...,
        6.89621914e-03, 6.92033179e-03, 6.94444444e-03],
       [3.61689815e-05, 7.23379630e-05, 1.08506944e-04, ...,
        1.03443287e-02, 1.03804977e-02, 1.04166667e-02],
       ...,
       [3.44810957e-03, 6.89621914e-03, 1.03443287e-02, ...,
        9.86159336e-01, 9.89607446e-01, 9.93055556e-01],
       [3.46016590e-03, 6.92033179e-03, 1.03804977e-02, ...,
        9.89607446e-01, 9.93067612e-01, 9.96527778e-01],
       [3.47222222e-03, 6.94444444e-03, 1.04166667e-02, ...,
        9.93055556e-01, 9.96527778e-01, 1.00000000e+00]]),
       array([[1.19560019e-06, 2.39120038e-06, 3.58680057e-06, ...,
        1.46819703e-03, 1.46939264e-03, 1.47058824e-03],
       [2.39120038e-06, 4.78240077e-06, 7.17360115e-06, ...,
        2.93639407e-03, 2.93878527e-03, 2.94117647e-03],
       [3.58680057e-06, 7.17

In [15]:
ppi_masks['position_landscape'] = position_landscape
ppi_masks

,Uniprot ID A,Uniprot ID B,Domain_id_a,Domain_id_b,Domain positions A,Domain positions B,Sites Masks,PSSM_masks,position_landscape
0,P53081,P53081,[PF01784],[PF01784],"[(12, 275)]","[(12, 275)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[118.0, -11.0, -15.0, 5.0, 33.0, 11.0, -8.0, ...","[[1.2056327160493826e-05, 2.4112654320987653e-..."
1,Q12188,P47037,[PF04825],[PF02463],"[(11, 136)]","[(2, 1208)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[174.0, 68.0, 113.0, 28.0, -10.0, 152.0, 14.0...","[[1.1956001912960306e-06, 2.391200382592061e-0..."
2,Q03020,P25374,[PF01592],[PF00266],"[(34, 161)]","[(100, 462)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[81.0, 82.0, 6.0, -8.0, 43.0, 56.0, 8.0, -9.0...","[[1.2194378391561491e-05, 2.4388756783122982e-..."
3,P17065,P51996,[PF06428],[PF00071],"[(79, 170)]","[(15, 176)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[114.0, -7.0, -27.0, -40.0, -32.0, -22.0, -28...","[[5.934788543484196e-06, 1.1869577086968392e-0..."
4,P00546,P20437,[PF00069],[PF00134],"[(8, 295)]","[(41, 193)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[164.0, -40.0, 3.0, -38.0, -37.0, 120.0, -32....","[[6.145979300341716e-06, 1.2291958600683432e-0..."
...,...,...,...,...,...,...,...,...,...
1601,Q12403,P54837,[PF01105],[PF01105],"[(23, 218)]","[(20, 206)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[148.0, 17.0, 28.0, 80.0, 33.0, 56.0, 43.0, 8...","[[2.1063717746182202e-05, 4.2127435492364404e-..."
1602,Q06549,Q06549,[PF00383],[PF00383],"[(8, 115)]","[(8, 115)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[111.0, -20.0, 32.0, -3.0, -26.0, 134.0, -35....","[[4.959333465582226e-05, 9.918666931164451e-05..."
1603,Q12335,Q12335,[PF03358],[PF03358],"[(8, 148)]","[(8, 148)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[206.0, 18.0, 25.0, 140.0, 75.0, 141.0, 132.0...","[[2.5507601265177026e-05, 5.101520253035405e-0..."
1604,P53725,P38792,[PF10175],[PF15985],"[(9, 186)]","[(197, 239)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[71.0, -10.0, -17.0, 61.0, 70.0, 10.0, 70.0, ...","[[1.4975888819001409e-05, 2.9951777638002818e-..."


## Prot2Vec

In [16]:
ppi_masks.to_pickle('./data/data_processed_2020-12-02.pkl')

In [17]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ppi_masks, test_size=0.1, random_state=86)

In [18]:

# for UA, UB, site_mask, PSSM_mask, position_landscape in zip(ppi_masks['Uniprot ID A'], ppi_masks['Uniprot ID B'], ppi_masks['Sites Masks'], ppi_masks['PSSM_masks'], ppi_masks['position_landscape']):
#     features = np.stack([PSSM_mask, position_landscape])
#     np.save(f'{processed_data_path}masks/{UA}_{UB}.npy', site_mask)
#     np.save(f'{processed_data_path}features/{UA}_{UB}.npy', features)


In [19]:
train_df

,Uniprot ID A,Uniprot ID B,Domain_id_a,Domain_id_b,Domain positions A,Domain positions B,Sites Masks,PSSM_masks,position_landscape
572,P39958,P07560,[PF00996],[PF00071],"[(1, 447)]","[(22, 182)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[99.0, 4.0, -29.0, 29.0, -1.0, 29.0, 62.0, -1...","[[1.031299953591502e-05, 2.062599907183004e-05..."
789,P53236,P61830,[PF01426],[PF00125],"[(368, 486)]","[(1, 132)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[140.0, 24.0, 37.0, 40.0, -14.0, -18.0, 33.0,...","[[7.92342799188641e-06, 1.584685598377282e-05,..."
1355,P53076,P38263,[PF00622],[PF09783],"[(401, 590)]","[(122, 347)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[71.0, 69.0, -27.0, -27.0, -3.0, 2.0, 60.0, -...","[[2.883539602532901e-06, 5.767079205065802e-06..."
1600,P05626,P07251,[PF05405],[PF02874],"[(75, 237)]","[(62, 129)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[80.0, 57.0, -15.0, 28.0, 17.0, 11.0, 7.0, 45...","[[7.519927808693037e-06, 1.5039855617386074e-0..."
262,P47122,Q06543,[PF03029],[PF03029],"[(8, 258)]","[(8, 255)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[151.0, -16.0, 34.0, 33.0, 3.0, 61.0, 115.0, ...","[[9.549274255156608e-06, 1.9098548510313215e-0..."
...,...,...,...,...,...,...,...,...,...
1001,Q03790,Q03790,[PF05172],[PF05172],"[(239, 357)]","[(239, 357)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[85.0, 24.0, -39.0, 80.0, 2.0, -11.0, -13.0, ...","[[4.432132963988919e-06, 8.864265927977838e-06..."
27,P33332,P32855,[PF09763],[PF04048],"[(640, 1316)]","[(34, 175)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[69.0, -30.0, 19.0, 74.0, 6.0, -16.0, 13.0, 2...","[[7.028197126873015e-07, 1.405639425374603e-06..."
1123,P40568,P39731,[PF08202],[PF05859],"[(223, 572)]","[(11, 146)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[69.0, -10.0, 12.0, -3.0, 10.0, 69.0, 10.0, -...","[[6.007304882737409e-06, 1.2014609765474818e-0..."
1452,P10622,P02400,[PF00428],[PF00428],"[(19, 105)]","[(17, 109)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[128.0, 3.0, 40.0, 109.0, 36.0, 38.0, 42.0, 9...","[[8.576329331046312e-05, 0.0001715265866209262..."


In [20]:
test_df

,Uniprot ID A,Uniprot ID B,Domain_id_a,Domain_id_b,Domain positions A,Domain positions B,Sites Masks,PSSM_masks,position_landscape
1482,P38779,P14120,[PF00687],[PF01248],"[(44, 287)]","[(8, 102)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[87.0, 18.0, -11.0, 16.0, -11.0, 42.0, -8.0, ...","[[2.5329280648429586e-05, 5.065856129685917e-0..."
811,P32325,P32562,[PF07535],[PF00069],"[(656, 700)]","[(82, 337)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[83.0, -7.0, 37.0, -24.0, 2.0, 38.0, -24.0, 7...","[[2.014829142488717e-06, 4.029658284977434e-06..."
235,P87108,P32830,[PF02953],[PF02953],"[(18, 82)]","[(21, 83)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[128.0, -19.0, 90.0, 100.0, 68.0, -23.0, -3.0...","[[9.864851533984414e-05, 0.0001972970306796883..."
155,P24279,P40359,[PF00493],[PF05916],"[(346, 569)]","[(69, 178)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[107.0, 17.0, 77.0, 1.0, 3.0, -19.0, 77.0, -4...","[[4.835052194388438e-06, 9.670104388776876e-06..."
1460,P19263,Q06213,[PF08638],[PF09748],"[(89, 292)]","[(19, 142)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[79.0, -8.0, -12.0, -31.0, -2.0, 23.0, -32.0,...","[[5.886716036591827e-06, 1.1773432073183654e-0..."
...,...,...,...,...,...,...,...,...,...
1370,Q08649,P38806,[PF01853],[PF12998],"[(223, 400)]","[(6, 111)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[123.0, -22.0, 6.0, -5.0, 70.0, 73.0, 128.0, ...","[[7.96876245119133e-06, 1.593752490238266e-05,..."
49,P22082,Q12406,[PF00176],[PF00022],"[(769, 1065)]","[(2, 467)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[121.0, -15.0, 35.0, -49.0, 14.0, -18.0, 15.0...","[[1.2310252840283087e-06, 2.4620505680566173e-..."
437,P11076,P41810,[PF00025],[PF01602],"[(5, 177)]","[(20, 560)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[174.0, 27.0, 9.0, 85.0, 49.0, -23.0, 56.0, -...","[[5.678172536950708e-06, 1.1356345073901415e-0..."
66,P00427,P04037,[PF02284],[PF01215],"[(45, 145)]","[(27, 155)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[80.0, 85.0, -10.0, 85.0, 16.0, -2.0, 0.0, 84...","[[4.359197907585005e-05, 8.71839581517001e-05,..."


In [29]:
for UA, UB, site_mask, PSSM_mask, position_landscape in zip(test_df['Uniprot ID A'], test_df['Uniprot ID B'], test_df['Sites Masks'], test_df['PSSM_masks'], test_df['position_landscape']):
    features = np.stack([PSSM_mask, position_landscape])
    np.save(f'{processed_data_path}test/masks/{UA}_{UB}.npy', np.expand_dims(site_mask, axis=0))
    np.save(f'{processed_data_path}test/features/{UA}_{UB}.npy', features)

In [30]:
for UA, UB, site_mask, PSSM_mask, position_landscape in zip(train_df['Uniprot ID A'], train_df['Uniprot ID B'], train_df['Sites Masks'], train_df['PSSM_masks'], train_df['position_landscape']):
    features = np.stack([PSSM_mask, position_landscape])
    np.save(f'{processed_data_path}train/masks/{UA}_{UB}.npy', np.expand_dims(site_mask, axis=0))
    np.save(f'{processed_data_path}train/features/{UA}_{UB}.npy', features)

In [28]:
np.expand_dims(test_df['Sites Masks'].iloc[0], axis=0)

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]])